In [1]:
PADDLE_SITEMAP = "https://developer.paddle.com/sitemap.xml"

In [2]:
!pip install -Uq pip

zsh:1: /Users/yjkim/Developer/WhatsUpDoc/whatsupdocc/venv/bin/pip: bad interpreter: /Users/yjkim/Developer/whatsupdocc/venv/bin/python3.11: no such file or directory


In [3]:
!pip install -q -r requirements.txt

zsh:1: /Users/yjkim/Developer/WhatsUpDoc/whatsupdocc/venv/bin/pip: bad interpreter: /Users/yjkim/Developer/whatsupdocc/venv/bin/python3.11: no such file or directory


In [4]:
from langchain.document_loaders.sitemap import SitemapLoader
# fixes a bug with asyncio and jupyter
import nest_asyncio

nest_asyncio.apply()

sitemap_loader = SitemapLoader(
    web_path=PADDLE_SITEMAP,
)

sitemap_loader.requests_per_second = 2

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

data = sitemap_loader.load_and_split(text_splitter=text_splitter)

Fetching pages: 100%|##########| 399/399 [01:46<00:00,  3.75it/s]


In [29]:
len(data)
# data[3000]

4186

In [10]:
from supabase.client import Client, create_client
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.supabase import SupabaseVectorStore
import os
from dotenv import load_dotenv

load_dotenv()

supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_SERVICE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

embeddings = OpenAIEmbeddings()

In [28]:
vector_store = SupabaseVectorStore.from_documents(
    documents=data[4100:],
    embedding=embeddings,
    client=supabase,
    table_name="paddle_documents",
    query_name="paddle_match_documents",
    chunk_size=100
)

2023-12-15 20:00:29,795:INFO - HTTP Request: POST https://qublpyarwoevdeqqicbz.supabase.co/rest/v1/paddle_documents "HTTP/1.1 201 Created"
